In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import scipy.io
import glob
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import scipy.io as sio
from torchvision import datasets, transforms


class CustomActivation(nn.Module):
    def forward(self, x):
        # Apply your custom activation function
        # Replace this line with your desired custom activation function
        return 8 * torch.tanh(x)

class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=(3, 2), stride=1, padding=0),
            nn.ReLU(),
        )

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1, 1, kernel_size=(3, 2), stride=1, padding=0),
            CustomActivation(),  # Use the custom activation function
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def encode(self, x):
        x = self.encoder(x)
        return x

def train(model, num_epochs=5, batch_size=64, learning_rate=1e-3):
    torch.manual_seed(42)
    criterion = nn.MSELoss()  # Mean Square Error loss
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=learning_rate,
                                 weight_decay=1e-5)

    training_data = dataset
    best_loss = float('inf')  # Initialize with a high value

    for epoch in range(num_epochs):
        for data in training_data:
            img = data
            recon = model(img)
            loss = criterion(recon, img)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Save the model if the current loss is the best so far
        if loss < best_loss:
            best_loss = loss
            torch.save(model.state_dict(), 'best_model.pth')

        print('Epoch:{}, Loss:{:.4f}'.format(epoch + 1, float(loss)))

    print('Training complete.')

In [ ]:
for i in range(1,10):
    # train autoencoder
    raw_data = scipy.io.loadmat(f'/content/drive/MyDrive/3dGrazNormalized/3Dnorm_2d_{i:02d}T_P.mat')['data']
    data = torch.empty(len(raw_data)*1000, 6, 7)

    counter = 0
    for d in raw_data:
        d = torch.tensor(np.array(d))
        for dd in torch.split(d, 1, dim=2):
            data[counter] = torch.squeeze(dd)
            counter += 1
    data = data.unsqueeze(1)
    print(data.shape)
    dataset = data.float()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(str(device))
    model = Autoencoder().to(device)
    dataset = dataset.to(device)
    max_epochs = 3
    train(model, num_epochs=max_epochs)


    # load best model and transform data
    model.load_state_dict(torch.load('best_model.pth'))
    file_list = glob.glob(f'/content/drive/MyDrive/3dGrazNormalized/3Dnorm_2d_{i:02d}*.mat')
    for f in file_list:

        trans = []

        for d in scipy.io.loadmat(f)['data']:
            d = torch.tensor(np.array(d))
            d = torch.split(d, 1, dim=2)
            trans_d = []

            for dd in d:
                dd = torch.squeeze(dd).float().unsqueeze(0).to(device)
                compressed_dd = torch.squeeze(model.encode(dd).cpu().detach())
                trans_d.append(np.array(compressed_dd).flatten())

            trans_d = np.transpose(np.array(trans_d), (1, 0))


            trans.append(trans_d)

        subject_data = {'data': trans, 'label': scipy.io.loadmat(f)['label']}
        sio.savemat('trans_'+ f.split('/')[-1], subject_data)

torch.Size([463000, 1, 6, 7])
cpu
Epoch:1, Loss:0.0306
Epoch:2, Loss:0.0306
Epoch:3, Loss:0.0306
Training complete.
torch.Size([464000, 1, 6, 7])
cpu
Epoch:1, Loss:0.0792
Epoch:2, Loss:0.0792
Epoch:3, Loss:0.0792
Training complete.
torch.Size([464000, 1, 6, 7])
cpu
Epoch:1, Loss:0.0517
Epoch:2, Loss:0.0517
Epoch:3, Loss:0.0517
Training complete.
torch.Size([464000, 1, 6, 7])
cpu
Epoch:1, Loss:0.0863
Epoch:2, Loss:0.0863
Epoch:3, Loss:0.0863
Training complete.
torch.Size([464000, 1, 6, 7])
cpu
Epoch:1, Loss:0.0448
Epoch:2, Loss:0.0448
Epoch:3, Loss:0.0448
Training complete.
torch.Size([464000, 1, 6, 7])
cpu
Epoch:1, Loss:0.0186
Epoch:2, Loss:0.0186
Epoch:3, Loss:0.0186
Training complete.
torch.Size([464000, 1, 6, 7])
cpu
Epoch:1, Loss:0.0441
Epoch:2, Loss:0.0441
Epoch:3, Loss:0.0441
Training complete.
torch.Size([464000, 1, 6, 7])
cpu
Epoch:1, Loss:0.0613
Epoch:2, Loss:0.0613
Epoch:3, Loss:0.0613
Training complete.
torch.Size([464000, 1, 6, 7])
cpu
Epoch:1, Loss:0.0840
Epoch:2, Loss:0.0

In [ ]:
import glob
import shutil

# Get a list of .mat files matching the pattern
file_list = glob.glob('trans*.mat')

# Specify the destination directory in Google Drive
destination_directory = '/content/drive/MyDrive/3dCompressedGraz/'

# Iterate over each .mat file
for file_path in file_list:
    # Copy the file to the destination directory
    shutil.copy(file_path, destination_directory)